<a href="https://colab.research.google.com/github/4L4N4F/Python_ETLpipeline/blob/main/proj005.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Jornada de Dados <Laboratória>**
**PROJ005**

A VAREJISTA, empresa líder no setor, enfrenta o desafio de gerenciar grandes volumes de dados dispersos. Para aprimorar a tomada de decisões, está sendo implementado um sistema ETL robusto com tabelas fato e dimensões. O projeto visa extrair, transformar e carregar dados de forma eficiente, estruturando-os hierarquicamente para facilitar análises estratégicas. Além de otimizar o armazenamento, a iniciativa busca ampliar a capacidade da empresa de identificar padrões e oportunidades de mercado, respondendo com agilidade às mudanças na demanda do consumidor.
<br><br>
[**DATASET**](https://github.com/4L4N4F/Python_ETLpipeline/blob/main/superstore.csv)
<br><br>
**DESCRIÇÃO DAS VARIÁVEIS**<BR>
<small>(Conforme fornecidas pela Laboratória na documentação do projeto.)</small>

| Variável | Descrição |
|----------------|------------------------------------------------------------------------------|
| category       | Categoria do produto adquirido                          |
| city           | Cidade onde o pedido foi feito                     |
| country        | País onde o hipermercado está localizado           |
| customer_id    | Identificador único para cada cliente                   |
| customer_name  | Nome do cliente        |
| discount       | Desconto aplicado no pedido            |
| market         | Mercado ou região onde o hipermercado atua        |
| unknown        | Coluna desconhecida ou não especificada                          |
| order_date     | Data do pedido                 |
| order_id       | Identificador único para cada pedido                   |
| order_priority | Nível de prioridade do pedido            |
| product_id     | Identificador exclusivo para cada produto        |
| product_name   | Nome do produto                                 |
| profit         | Lucro gerado pelo pedido                              |
| quantity       | Quantidade de produtos encomendados no pedido                    |
| region         | Região onde o pedido foi feito                    |
| row_id         | Identificador exclusivo para cada linha do conjunto de dados    |
| sales          | Valor total de vendas do produto no pedido           |
| segment        | Segmento do cliente|
| ship_date      | Data de envio do pedido                                |
| ship_mode      | Modo de envio do pedido        |
| shipping_cost  | Custo de envio do pedido                   |
| state          | Estado ou região do país                |
| sub_category   | Subcategoria do produto             |
| year           | Ano do pedido                  |
| market2        | Outra coluna relacionada a informações de mercado                               |
| weeknum        | Número da semana em que o pedido foi feito

In [6]:
import pandas as pd
df = pd.read_csv('https://github.com/4L4N4F/Python_ETLpipeline/raw/refs/heads/main/superstore.csv')
